Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [37]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [38]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-140888
aml-quickstarts-140888
southcentralus
d4ad7261-832d-46b2-b093-22156001df5b


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [39]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-1,quick-starts-ws-140888,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [40]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [41]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing Dataset"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00
mean,40.04,257.34,2.56,962.17,0.17,0.08,93.57,-40.52,3.62,5166.86
std,10.43,257.33,2.76,187.65,0.50,1.57,0.58,4.62,1.74,72.21
min,17.00,0.00,1.00,0.00,0.00,-3.40,92.20,-50.80,0.63,4963.60
25%,32.00,102.00,1.00,999.00,0.00,-1.80,93.08,-42.70,1.34,5099.10
50%,38.00,179.00,2.00,999.00,0.00,1.10,93.75,-41.80,4.86,5191.00
75%,47.00,318.00,3.00,999.00,0.00,1.40,93.99,-36.40,4.96,5228.10
max,98.00,4918.00,56.00,999.00,7.00,1.40,94.77,-26.90,5.04,5228.10


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [42]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.80,92.89,-46.20,1.30,5099.10,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.80,92.89,-46.20,1.31,5099.10,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.40,94.47,-41.80,4.97,5228.10,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [44]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             blocked_models = ['XGBoostClassifier'],
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [45]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [46]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [47]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [48]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [a4a3313c][21bca054-3198-475f-8970-88837530d7a2], (This step will run and generate new outputs)
Submitted PipelineRun 38253635-ac84-4a3d-ad22-e2238d4f531b
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/38253635-ac84-4a3d-ad22-e2238d4f531b?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-140888/workspaces/quick-starts-ws-140888


In [49]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [50]:
pipeline_run.wait_for_completion()

PipelineRunId: 38253635-ac84-4a3d-ad22-e2238d4f531b
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/38253635-ac84-4a3d-ad22-e2238d4f531b?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-140888/workspaces/quick-starts-ws-140888
PipelineRun Status: Running


StepRunId: 7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-140888/workspaces/quick-starts-ws-140888
StepRun( automl_module ) Status: NotStarted
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465', 'target': 'auto-ml', 'status': 'Completed', 'startTimeUtc': '2021-03-19T18:45:53.397459Z', 'endTimeUtc': '2021-03-19T19:14:36.495891Z

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [51]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465/metrics_data, 1 files out of an estimated total of 1


In [52]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_4,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_22,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_5,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_12,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_14,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_9,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_0,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_7,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_6,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_8,...,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_3,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_11,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_1,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_13,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_17,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_2,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_16,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_18,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_15,7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465_21
accuracy,[0.8992412746585736],[0.9132018209408195],[0.8995447647951441],[0.8916540212443096],[0.7341426403641882],[0.7687405159332321],[0.9116843702579667],[0.8880121396054628],[0.8880121396054628],[0.7141122913505311],...,[0.8880121396054628],[0.7207890743550834],[0.8904400606980273],[0.8880121396054628],[0.9031866464339909],[0.7939301972685888],[0.7174506828528073],[0.9119878603945372],[0.7402124430955994],[0.9110773899848255]
log_loss,[0.2411384950516903],[0.19851624229348674],[0.24710842097050564],[0.2553666128943903],[0.5269491495740212],[0.5224117673479711],[0.17775706110025447],[0.2715040731919684],[0.279499074618926],[0.5626134497051087],...,[0.2833043062813159],[0.5801942961856235],[0.23841730012952994],[0.24659598292618673],[0.22576647794112129],[0.5069475626940056],[0.5506915204995705],[0.17701928823797536],[0.576164504778481],[0.18092002426572737]
weighted_accuracy,[0.9768011702488849],[0.9572916184016419],[0.9774315713566799],[0.9842661863216413],[0.7329368884905176],[0.7533547330154156],[0.9514937218005303],[0.9843450583187134],[0.9843450583187134],[0.711321592289952],...,[0.9843450583187134],[0.7178407192666239],[0.9846844608483891],[0.9843450583187134],[0.9608893771952084],[0.7933305333391129],[0.7126702344556343],[0.9544760260746684],[0.7414290801053206],[0.9593465443545185]
norm_macro_recall,[0.17368717432902292],[0.47123166378622106],[0.1716606742280682],[0.037256852404477625],[0.4779983958417846],[0.6614235144402025],[0.5026785366965085],[0.0],[0.0],[0.45070547766311564],...,[0.0],[0.46532906545743513],[0.021680216802167918],[0.0],[0.3415402882668608],[0.5926910772867129],[0.4734109849642585],[0.4817059277906517],[0.470624084231273],[0.4333153652794217]
recall_score_weighted,[0.8992412746585736],[0.9132018209408195],[0.8995447647951441],[0.8916540212443096],[0.7341426403641882],[0.7687405159332321],[0.9116843702579667],[0.8880121396054628],[0.8880121396054628],[0.7141122913505311],...,[0.8880121396054628],[0.7207890743550834],[0.8904400606980273],[0.8880121396054628],[0.9031866464339909],[0.7939301972685888],[0.7174506828528073],[0.9119878603945372],[0.7402124430955994],[0.9110773899848255]
f1_score_weighted,[0.8723981505657483],[0.9086436856906703],[0.8722950091956113],[0.8450656018685619],[0.7805664383525297],[0.809253394768304],[0.9091539479147899],[0.8353395018439429],[0.8353395018439429],[0.7651476903332187],...,[0.8353395018439429],[0.7703805516072585],[0.841167904412042],[0.8353395018439429],[0.8925206884427337],[0.826761902234856],[0.7679658226270446],[0.9082853054497467],[0.7849397125495022],[0.9047821362852003]
AUC_weighted,[0.9018708078399991],[0.945106669111804],[0.8976927722113858],[0.8978127136021872],[0.842530846702862],[0.8989528514560607],[0.9450464668693167],[0.8892394511778338],[0.8424613825769154],[0.8277641628090922],...,[0.8331587468301204],[0.8594833350930913],[0.9025094147045366],[0.8940357175273734],[0.921386059383492],[0.8929520771626035],[0.8667085303799039],[0.9461967927949957],[0.8179238747274691],[0.9457290676802872]
AUC_macro,[0.9018708078399991],[0.945106669111804],[0.8976927722113859],[0.8978127136021872],[0.842530846702862],[0.8989528514560607],[0.9450464668693166],[0.8892394511778338],[0.84246

### Retrieve the Best Model

In [53]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/7fbd7d05-9d4f-44d5-bc05-a3ec9abdb465/model_data, 1 files out of an estimated total of 1


In [54]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [55]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('18',
                                             Pipeline(memory=None,
                                                      steps=[('sparsenormalizer',
                                                              <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7f7757b7d7f0>),
                                                             ('lightgbmclassifier',
                                                              LightGBMClassifier(boosting_t

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [56]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [57]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [58]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,28650,608
1,1539,2153


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [59]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,239e1846-4c68-4dca-b767-63a949fac98b,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [60]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [61]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [62]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  ce3c7fa9-043a-4bb9-afe3-3a9070f4cbaa


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [63]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …